# Simple 2 bus Example

This tutorial shows how to perform three phase load flow in an unbalanced system.
<img src="pics/Two bus system.png" width="80%">

Three phase or unbalanced load flow is performed using the function 
 pp.runpp_3ph(net) 

In [3]:
import pandapower as pp

# Create Buses
After this import we can start by creating an empty network and two 110 kV buses 

# Create External Grid
For negative and zero sequence networks. 
The following data is needed in addition to a voltage reference.
- s_sc_max_mva
- rx_max
- r0x0_max
- x0x_max


In [4]:
net = pp.create_empty_network(sn_mva=100)
bus_eg = pp.create_bus(net, vn_kv=110, name="Ext. Grid")
bus_load = pp.create_bus(net, vn_kv=110, name="Load")

pp.create_ext_grid(
    net,
    bus=bus_eg,
    vm_pu=1.0,
    name="Grid Connection",
    s_sc_max_mva=5000,
    rx_max=0.1,
    r0x0_max=0.1,
    x0x_max=1.0,
)

0

# Creating Lines and transformers

Just like balanced load flow there are two ways of creating lines and transformers:
- Create using a Standard type from standard type library/ Customized standard type
- Create directly from parameters

# Create element using a Customized Standard type 

**This is useful for large networks with many lines and transformers**


In [5]:
# Line Type created
pp.create_std_type(
    net,
    {
        "r0_ohm_per_km": 0.0848,
        "x0_ohm_per_km": 0.4649556,
        "c0_nf_per_km": 230.6,
        "max_i_ka": 0.963,
        "r_ohm_per_km": 0.0212,
        "x_ohm_per_km": 0.1162389,
        "c_nf_per_km": 230,
    },
    name="example_type",
)

After a Standard type has been created with zero sequence parameters, there are two more steps required:
- Create a line using **pp.create_line** with the standard type
- Use **pp.add_zero_impedance_parameters** function to insert them into network elements line and trafo from standard types



In [6]:
# Actual line created with the length of the line in km
pp.create_line(
    net, from_bus=bus_eg, to_bus=bus_load, length_km=50.0, std_type="example_type"
)

0

# Create Unbalanced Load
We can create three phase loads by specifying P, Q values in each phase.
The default values are set to zero.

Connection type of load can be specified while creating a three phase load

**'wye' (Default)** - 3 phase phase-earth load

**'delta'** - 3 phase delta load

In [7]:
# This function creates an unbalanced load
pp.create_asymmetric_load(
    net, bus_load, p_a_mw=50, q_a_mvar=50, p_b_mw=10, q_b_mvar=15, p_c_mw=10, q_c_mvar=5
)

0

# add_zero_impedance_parameters

This function is required when lines and transformers are created using standard types.

This adds the zero sequence parameters from standard type to the network elements trafo and line

# runpp_3ph

This function performs the calculation and returns results in result tables:

-res_element_3ph 

In [8]:
pp.add_zero_impedance_parameters(net)
pp.runpp_3ph(net)
print(net.trafo)
print("----------------")
net.res_bus_3ph

Empty DataFrame
Columns: [name, std_type, hv_bus, lv_bus, sn_mva, vn_hv_kv, vn_lv_kv, vk_percent, vkr_percent, pfe_kw, i0_percent, shift_degree, tap_side, tap_neutral, tap_min, tap_max, tap_step_percent, tap_step_degree, tap_pos, tap_phase_shifter, parallel, df, in_service, vector_group, vk0_percent, vkr0_percent, mag0_percent, mag0_rx, si0_hv_partial, xn_ohm]
Index: []

[0 rows x 30 columns]
----------------


,vm_a_pu,va_a_degree,vm_b_pu,va_b_degree,vm_c_pu,va_c_degree,p_a_mw,q_a_mvar,p_b_mw,q_b_mvar,p_c_mw,q_c_mvar,unbalance_percent
0,0.967429,-1.051253,1.013028,-119.470654,1.019784,120.471400,-55.707739,-60.797345,-8.779937,0.880937,-9.373928,11.441666,1.985821
1,0.749574,-9.010213,1.091380,-124.105792,1.051243,125.894504,50.000000,50.000000,10.000000,15.000000,10.000000,5.000000,6.642212


The convergence of load flow can be checked with the following:

In [9]:
net["converged"]

True

# 3 Bus system with a transformer in between External Grid and Distribution system

<img src="pics/Three bus system.png" width="80%">

In [10]:
net = pp.create_empty_network(sn_mva=100)

bus_eg = pp.create_bus(net, 10)
bus_lv = pp.create_bus(net, 0.4)
bus_load = pp.create_bus(net, 0.4)

pp.create_ext_grid(net, bus_eg, s_sc_max_mva=10000, rx_max=0.1)
net.ext_grid["r0x0_max"] = 0.1
net.ext_grid["x0x_max"] = 1.0
print(net.trafo)
print("-----------------------------------")

Empty DataFrame
Columns: [name, std_type, hv_bus, lv_bus, sn_mva, vn_hv_kv, vn_lv_kv, vk_percent, vkr_percent, pfe_kw, i0_percent, shift_degree, tap_side, tap_neutral, tap_min, tap_max, tap_step_percent, tap_step_degree, tap_pos, tap_phase_shifter, parallel, df, in_service]
Index: []

[0 rows x 23 columns]
-----------------------------------


# Create line and transformer using parameters.

Creates a transformer using parameters

In [11]:
pp.create_transformer_from_parameters(
    net,
    hv_bus=bus_eg,
    lv_bus=bus_lv,
    sn_mva=1.6,
    vn_hv_kv=10,
    vn_lv_kv=0.4,
    vk_percent=6,
    vkr_percent=0.78125,
    pfe_kw=2.7,
    i0_percent=0.16875,
    shift_degree=0,
    tap_side="lv",
    tap_neutral=0,
    tap_min=-2,
    tap_max=2,
    tap_step_degree=0,
    tap_step_percent=2.5,
    tap_changer_type="Ratio",
    vk0_percent=6,
    vkr0_percent=0.78125,
    mag0_percent=100,
    mag0_rx=0.0,
    si0_hv_partial=0.9,
    vector_group="Dyn",
    parallel=1,
    tap_pos=0,
    index=pp.get_free_id(net.trafo) + 1,
)
pp.create_line_from_parameters(
    net,
    bus_lv,
    bus_load,
    length_km=0.5,
    r_ohm_per_km=0.1941,
    x_ohm_per_km=0.07476991,
    c_nf_per_km=1160.0,
    max_i_ka=0.421,
    endtemp_degree=70.0,
    r0_ohm_per_km=0.7766,
    x0_ohm_per_km=0.2990796,
    c0_nf_per_km=496.2,
    index=pp.get_free_id(net.line) + 1,
)

1

 Reset index of transformer to prevent KeyError

In [12]:
net.trafo.reset_index(inplace=True, drop=True)

# Create  Asymmetric load

Just like before create three phase load with three phase power values

In [13]:
# Creates a 3 phase wye load
pp.create_asymmetric_load(
    net,
    bus_load,
    p_a_mw=0.0300,
    q_a_mvar=0.0048,
    p_b_mw=0.0280,
    q_b_mvar=0.0036,
    p_c_mw=0.027,
    q_c_mvar=0.0043,
    type="wye",
)

0

# Run Asymmetric load flow for the network

Same as before, add zero sequence parameters and run 3 phase load flow.


In [14]:
pp.add_zero_impedance_parameters(net)
pp.runpp_3ph(net)
net.res_bus_3ph

,vm_a_pu,va_a_degree,vm_b_pu,va_b_degree,vm_c_pu,va_c_degree,p_a_mw,q_a_mvar,p_b_mw,q_b_mvar,p_c_mw,q_c_mvar,unbalance_percent
0,1.000000,-0.000019,1.000000,-119.999989,1.000000,120.000007,-0.031954,-0.005008,-0.030429,-0.005846,-0.030466,-0.004107,0.000033
1,0.998897,-0.204155,0.999061,-120.187909,0.999050,119.819285,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011346
2,0.928733,-0.903632,0.945275,-121.393616,0.948877,119.498843,0.030000,0.004800,0.028000,0.003600,0.027000,0.004300,0.214209
